### 予測モデルの仕組みをわかりやすく説明します！

#### どんなことを予測するの？
このモデルは、**「翌月末の限界利益がどのくらいになりそうか」**を、日次の次月データを使って予測するものです。

#### 例えば？
- **2月12日**に、翌月（3月）のために積み上げている「次月確定」「次月A」「次月B」「次月C」「次月D」のデータがあります
- これらを使って、**3月末**の限界利益を予測します
- また、**3月のスタートMP**（次月確定 + 次月A）がいくらになるかも分かります

#### どうやって予測するの？

1.  **日次の次月データを使うよ！**
    *   2月の各日における「次月確定」「次月A」「次月B」「次月C」「次月D」の値を使います
    *   これらのデータから、「3月末にどれくらいの限界利益があるか」のヒントを探すんだ

2.  **２つの限界利益を足し算するよ！**
    翌月末の限界利益を予測するとき、大きく分けて２つの部分に分けて考えます

    *   **「3月のスタートMP（確定部分）」**:
        これは、「次月確定MP」と「次月AMP（A）」を合わせた限界利益です。これは3月スタート時点でほぼ確実に手に入る限界利益だね！

    *   **「3月中に増えそうな限界利益（追加部分）」**:
        これは、「次月B」「次月C」「次月D」が3月中にどれくらい追加MPに変わるかを予測します
        この部分を予測するために、過去のデータから学んだ「転換率」を使います

3.  **合計するよ！**
    最後に、「3月のスタートMP」と「3月中の追加分」を足し合わせると、3月末の限界利益の予測が出てきます

#### まとめると...

このモデルは、当月の日次データ（次月確定、次月A、次月B、次月C、次月D）を使って、**「翌月のスタートMP」**と**「翌月末の限界利益」**を教えてくれる賢い予測屋さんなんです！

**当月末MP予測モデルとの違い**: 当月データではなく、**次月データ**を使う点だけが違います。予測ロジックは同じです！

# MP Next Month-End Prediction Model (as of Current Month-End)
## 当月末時点での翌月末MP予測モデル

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded!')

## 2. データの読み込み

In [ ]:
try:
    from google.colab import files
    print('Upload CSV file:')
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
except:
    filename = 'progress_data.csv'
    print(f'Using: {filename}')

In [ ]:
try:
    df = pd.read_csv(filename, encoding='utf-8-sig')
except:
    df = pd.read_csv(filename, encoding='shift_jis')

print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head()

## 3. データ前処理（日次の次月データを使用）

In [ ]:
# 元のカラム名を保存
original_columns = df.columns.tolist()
print(f'Original columns: {original_columns}')

# 日付処理
df['date'] = pd.to_datetime(df.iloc[:, 0], errors='coerce')
# NaT（無効な日付）を含む行を削除
df = df.dropna(subset=['date']).copy()

# 年月情報を追加
df['year_month'] = df['date'].dt.to_period('M')
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

print(f'\nDaily data points: {len(df)}')

In [ ]:
# カラム名を変更（次月の確定、A、B、C、Dに対応）
# 13列フォーマットの場合は列8-12、12列フォーマットの場合は列7-11、7列フォーマットの場合は列2-6を使用
if len(original_columns) >= 13:
    # 13列フォーマット: 次月データを使用（列8-12）
    col_mapping = {
        original_columns[1]: 'Current_MP',     # 当月MP（参考用）
        original_columns[7]: 'Next_MP',        # 次月MP（参考用）
        original_columns[8]: 'Next_Kakutei',   # 次月確定MP
        original_columns[9]: 'Next_A',         # 次月AMP
        original_columns[10]: 'Next_B',        # 次月BMP
        original_columns[11]: 'Next_C',        # 次月CMP
        original_columns[12]: 'Next_D'         # 次月DMP
    }
    print('Using 13-column format (next month data from columns 8-12)')
elif len(original_columns) >= 12:
    # 12列フォーマット: 次月データを使用（列7-11）
    col_mapping = {
        original_columns[1]: 'Current_MP',     # 当月MP（参考用）
        original_columns[7]: 'Next_Kakutei',   # 次月確定MP
        original_columns[8]: 'Next_A',         # 次月AMP
        original_columns[9]: 'Next_B',         # 次月BMP
        original_columns[10]: 'Next_C',        # 次月CMP
        original_columns[11]: 'Next_D'         # 次月DMP
    }
    print('Using 12-column format (next month data from columns 7-11)')
else:
    # 7列フォーマット: 次月データのみ（列2-6）
    col_mapping = {
        original_columns[1]: 'Current_MP',
        original_columns[2]: 'Next_Kakutei',
        original_columns[3]: 'Next_A',
        original_columns[4]: 'Next_B',
        original_columns[5]: 'Next_C',
        original_columns[6]: 'Next_D'
    }
    print('Using 7-column format (next month data only)')

print(f'Mapping: {col_mapping}')

df = df.rename(columns=col_mapping)

# 数値変換
for col in ['Current_MP', 'Next_MP', 'Next_Kakutei', 'Next_A', 'Next_B', 'Next_C', 'Next_D']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f'\nColumns after mapping: {df.columns.tolist()}')
df.head()

In [ ]:
# 翌月の月末MPを取得（ターゲット変数として使用）
# 当月末MP予測モデルと同じロジック：
# - 2月のデータには「次月（3月）データ」が含まれる
# - 3月末のMPを取得して、2月の各日から予測する

# 各月の翌月の月末MPを取得
# 例：2月のデータ → 3月末のMPを取得
def get_next_month_end_mp(df):
    result = {}
    for ym in df['year_month'].unique():
        # 翌月のyear_month
        next_ym = ym + 1
        # 翌月末のMP（翌月の最後の日のCurrent_MP）
        next_month_data = df[df['year_month'] == next_ym]
        if len(next_month_data) > 0:
            result[ym] = next_month_data['Current_MP'].iloc[-1]
    return result

next_month_end_mp_dict = get_next_month_end_mp(df)

# 日次データに翌月末MPを追加
df['Next_Month_End_MP'] = df['year_month'].map(next_month_end_mp_dict)

# 月末日を特定
month_end_dates = df.groupby('year_month')['date'].max()
df['is_month_end'] = df.apply(lambda x: x['date'] == month_end_dates[x['year_month']], axis=1)

# 訓練データ作成（月末以外の日のデータを使用）
train_data = df[~df['is_month_end']].copy()

print(f'Training data points: {len(train_data)} days')
print(f'Unique months: {train_data["year_month"].nunique()}')
print(f'Months with next month end MP: {train_data["Next_Month_End_MP"].notna().sum()} days')

if len(train_data[train_data['Next_Month_End_MP'].notna()]) > 0:
    # 翌月末MPがあるデータのみで訓練
    train_data = train_data[train_data['Next_Month_End_MP'].notna()].copy()
    
    # 翌月のスタートMP（確定部分）
    train_data['Next_Month_Start_MP'] = train_data['Next_Kakutei'] + train_data['Next_A']
    
    # 翌月に追加で増える部分
    train_data['Additional'] = train_data['Next_Month_End_MP'] - train_data['Next_Month_Start_MP']
    
    # 次月のBCD合計
    train_data['Next_BCD_Total'] = train_data['Next_B'] + train_data['Next_C'] + train_data['Next_D']
    
    # 転換率計算
    train_data['Conversion_Rate'] = np.where(
        train_data['Next_BCD_Total'] > 0,
        train_data['Additional'] / train_data['Next_BCD_Total'] * 100,
        0
    )
    
    print('Calculations complete!')
    print(train_data[['date', 'year_month', 'Next_Month_Start_MP', 'Next_BCD_Total', 'Next_Month_End_MP']].head(10))
else:
    print('ERROR: No training data with Next_Month_End_MP!')

## 4. 月別転換率の分析

In [ ]:
# 月別転換率
monthly_stats = train_data.groupby('month').agg({
    'Conversion_Rate': ['mean', 'std', 'count']
}).round(1)
monthly_stats.columns = ['Avg_Rate', 'Std', 'Count']

print('Monthly BCD Conversion Rate')
print('='*50)
month_names = ['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for m in range(1, 13):
    if m in monthly_stats.index:
        r = monthly_stats.loc[m]
        print(f'{m:2d} ({month_names[m]}): {r["Avg_Rate"]:5.1f}% (n={int(r["Count"])})')
        
display(monthly_stats)

In [ ]:
# 月内の日による転換率の変化を可視化
fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for m in range(1, 13):
    month_data = train_data[train_data['month'] == m]
    if len(month_data) > 0:
        daily_avg = month_data.groupby('day')['Conversion_Rate'].mean()
        axes[m-1].plot(daily_avg.index, daily_avg.values, marker='o')
        axes[m-1].set_title(f'{month_names[m]} - Conversion Rate by Day')
        axes[m-1].set_xlabel('Day of Month')
        axes[m-1].set_ylabel('Rate (%)')
        axes[m-1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. モデル構築

In [ ]:
class NextMonthMPPredictor:
    def __init__(self):
        self.model = None
        self.monthly_rates = {}
        self.overall_avg_rate = None

    def train(self, data):
        # 月別平均転換率を計算
        for m in range(1, 13):
            sub = data[data['month'] == m]
            if len(sub) > 0:
                self.monthly_rates[m] = sub['Conversion_Rate'].mean()
        self.overall_avg_rate = data['Conversion_Rate'].mean()

        # Random Forestモデルの訓練
        # 特徴量: Next_B、Next_C、Next_D、月、日
        X = data[['Next_B', 'Next_C', 'Next_D', 'month', 'day']]
        y = data['Additional']
        
        self.model = RandomForestRegressor(
            n_estimators=100, 
            max_depth=7,
            random_state=42
        )
        self.model.fit(X, y)

        # モデル評価
        pred_additional = self.model.predict(X)
        pred = data['Next_Month_Start_MP'] + pred_additional
        
        return {
            'mae': mean_absolute_error(data['Next_Month_End_MP'], pred),
            'r2': r2_score(data['Next_Month_End_MP'], pred)
        }

    def predict(self, next_kakutei, next_a, next_b, next_c, next_d, month, day):
        """日次の次月データから翌月末のMPを予測"""
        # 翌月のスタートMP
        next_month_start = next_kakutei + next_a
        next_bcd = next_b + next_c + next_d

        # Random Forestによる予測
        add_rf = self.model.predict([[next_b, next_c, next_d, month, day]])[0]
        
        # 月別転換率による予測
        rate = self.monthly_rates.get(month, self.overall_avg_rate) / 100
        add_rate = next_bcd * rate
        
        # ハイブリッド予測（RF 70% + 転換率 30%）
        additional = add_rf * 0.7 + add_rate * 0.3

        return {
            'month': month,
            'day': day,
            'next_month_start': next_month_start,
            'additional': additional,
            'forecast': next_month_start + additional,
            'next_bcd': next_bcd,
            'rate': rate * 100
        }

print('NextMonthMPPredictor defined!')

In [ ]:
predictor = NextMonthMPPredictor()
metrics = predictor.train(train_data)

print('Model Trained!')
print(f'MAE: {metrics["mae"]/10000:,.0f} (10K JPY)')
print(f'R2:  {metrics["r2"]:.3f}')

## 6. 予測実行

In [ ]:
def show_prediction(p):
    mn = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    next_month_idx = (p['month'] % 12) + 1
    print('='*70)
    print(f'📅 Current Month: {mn[p["month"]]} Day {p["day"]}')
    print(f'🎯 Predicting: {mn[next_month_idx]} End MP')
    print('='*70)
    print(f'\n💰 {mn[next_month_idx]} Start MP:')
    print(f'   Next Kakutei + Next A = {p["next_month_start"]/10000:>10,.0f} (10K JPY)')
    print(f'   ↑ This will be the MP at {mn[next_month_idx]} start')
    print(f'\n📈 Additional MP (predicted growth during {mn[next_month_idx]}):')
    print(f'   From B, C, D conversion = {p["additional"]/10000:>10,.0f} (10K JPY)')
    print(f'   Conversion Rate: {p["rate"]:.1f}%')
    print(f'\n🎯 {mn[next_month_idx]} END FORECAST:')
    print(f'   Start MP + Additional = {p["forecast"]/10000:>10,.0f} (10K JPY)')
    print('='*70)

In [ ]:
# 最新データで予測
latest = df[~df['is_month_end']].iloc[-1]
print(f'Latest Date: {latest["date"].strftime("%Y-%m-%d")}')
print(f'Current Month: {latest["year_month"]}')
print()

p = predictor.predict(
    latest['Next_Kakutei'], latest['Next_A'],
    latest['Next_B'], latest['Next_C'], latest['Next_D'],
    month=int(latest['month']),
    day=int(latest['day'])
)
show_prediction(p)

# 実際の翌月末MPと比較（データがあれば）
if pd.notna(latest.get('Next_Month_End_MP')):
    actual = latest['Next_Month_End_MP']
    print(f'\nActual Next Month-End MP: {actual/10000:>10,.0f} (10K)')
    print(f'Prediction Error: {abs(p["forecast"] - actual)/10000:>10,.0f} (10K)')
else:
    print(f'\nNext month data not yet available (future prediction)')

In [ ]:
# カスタム予測例
# 例: 2月12日時点の次月データから3月末を予測
p = predictor.predict(
    next_kakutei=500000,
    next_a=45000000,
    next_b=4500000,
    next_c=3000000,
    next_d=15000000,
    month=2,  # 2月
    day=12    # 12日
)
show_prediction(p)

## 7. 月内での翌月予測の推移

In [ ]:
# 最新月のデータで、日ごとに翌月末の予測がどう変化するかシミュレーション
latest_month = df[df['year_month'] == df['year_month'].max()].copy()
latest_month = latest_month[~latest_month['is_month_end']]

if len(latest_month) > 0:
    simulation_results = []
    
    for _, row in latest_month.iterrows():
        p = predictor.predict(
            row['Next_Kakutei'], row['Next_A'],
            row['Next_B'], row['Next_C'], row['Next_D'],
            month=int(row['month']),
            day=int(row['day'])
        )
        simulation_results.append({
            'date': row['date'],
            'day': row['day'],
            'next_month_start': p['next_month_start'],
            'additional': p['additional'],
            'forecast': p['forecast'],
            'actual': row.get('Next_Month_End_MP', None)
        })
    
    sim_df = pd.DataFrame(simulation_results)
    
    current_ym = latest_month.iloc[0]['year_month']
    next_month_idx = (int(latest_month.iloc[0]['month']) % 12) + 1
    mn = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    next_ym_display = mn[next_month_idx]
    
    # 可視化
    plt.figure(figsize=(14, 6))
    plt.plot(sim_df['day'], sim_df['forecast']/10000, 
             label=f'Predicted {next_ym_display} Month-End MP', marker='o', linewidth=2, color='steelblue')
    
    # 実績値があれば表示
    if pd.notna(sim_df['actual'].iloc[0]):
        plt.axhline(y=sim_df['actual'].iloc[0]/10000, 
                    color='red', linestyle='--', label=f'Actual {next_ym_display} Month-End MP', linewidth=2)
    
    plt.fill_between(sim_df['day'], 
                     sim_df['next_month_start']/10000, 
                     sim_df['forecast']/10000, 
                     alpha=0.3, label='Additional (Predicted)', color='lightblue')
    
    plt.title(f'Daily Prediction Progress: {current_ym} → {next_ym_display} End')
    plt.xlabel(f'Day of {current_ym}')
    plt.ylabel('MP (10K JPY)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f'\nDaily prediction for {next_ym_display} End from each day of {current_ym}:')
    display(sim_df[['day', 'next_month_start', 'additional', 'forecast', 'actual']].head(10))
else:
    print('No data available for simulation')

## 8. 翌月スタートMPの推移分析（予測版）

このセクションでは、**「月末までに次月確定+次月Aがどれだけ積み上がるか」を予測**します。

**重要**：これは実績値の推移ではなく、**予測**です！
- 例：2月12日時点で、「2月末の次月確定+次月A」を予測
- これにより、3月のスタートMPの見通しが立てられます

In [ ]:
# 翌月スタートMP予測モデルの構築と実行
# 目的：月末までに「次月確定 + 次月A」がどれだけ積み上がるかを予測

# 1. 訓練データの準備
# 各月の月末における「次月確定 + 次月A」を取得
month_end_next_start_mp = df[df['is_month_end']].groupby('year_month').last()
month_end_next_start_mp['Month_End_Next_Start_MP'] = month_end_next_start_mp['Next_Kakutei'] + month_end_next_start_mp['Next_A']

# 月末以外の日のデータに、その月の月末の「次月確定+次月A」を追加
df['Month_End_Next_Start_MP'] = df['year_month'].map(
    month_end_next_start_mp['Month_End_Next_Start_MP'].to_dict()
)

# 訓練データ作成（月末以外の日で、ターゲット変数がある日）
train_data_start = df[~df['is_month_end'] & df['Month_End_Next_Start_MP'].notna()].copy()

print(f'Training data for next month start MP prediction: {len(train_data_start)} days')

if len(train_data_start) > 0:
    # 2. 予測モデルの構築（当月末MP予測モデルと同じロジック）
    class NextMonthStartMPPredictor:
        def __init__(self):
            self.model = None
            self.monthly_avg = {}
            self.overall_avg = None

        def train(self, data):
            # 月別平均成長率を計算
            for m in range(1, 13):
                sub = data[data['month'] == m]
                if len(sub) > 0:
                    # 現在の次月確定+Aから月末までの成長
                    current_start = sub['Next_Kakutei'] + sub['Next_A']
                    growth = sub['Month_End_Next_Start_MP'] - current_start
                    self.monthly_avg[m] = growth.mean()
            
            # Random Forestモデルの訓練
            X = data[['Next_Kakutei', 'Next_A', 'month', 'day']]
            y = data['Month_End_Next_Start_MP']
            
            self.model = RandomForestRegressor(
                n_estimators=100, 
                max_depth=7,
                random_state=42
            )
            self.model.fit(X, y)
            
            # モデル評価
            pred = self.model.predict(X)
            
            return {
                'mae': mean_absolute_error(y, pred),
                'r2': r2_score(y, pred)
            }

        def predict(self, next_kakutei, next_a, month, day):
            """日次の次月データから月末の次月スタートMPを予測"""
            current_value = next_kakutei + next_a
            
            # Random Forestによる予測
            pred_rf = self.model.predict([[next_kakutei, next_a, month, day]])[0]
            
            # 月別平均による予測
            growth = self.monthly_avg.get(month, 0)
            pred_avg = current_value + growth
            
            # ハイブリッド予測
            forecast = pred_rf * 0.7 + pred_avg * 0.3
            
            return {
                'month': month,
                'day': day,
                'current': current_value,
                'forecast': forecast,
                'growth': forecast - current_value
            }

    # 3. モデルの訓練
    start_predictor = NextMonthStartMPPredictor()
    metrics_start = start_predictor.train(train_data_start)
    
    print(f'Next Month Start MP Predictor Trained!')
    print(f'MAE: {metrics_start["mae"]/10000:,.0f} (10K JPY)')
    print(f'R2:  {metrics_start["r2"]:.3f}')
    
    # 4. 最新月のシミュレーション
    latest_month_for_start = df[df['year_month'] == df['year_month'].max()].copy()
    latest_month_for_start = latest_month_for_start[~latest_month_for_start['is_month_end']]
    
    if len(latest_month_for_start) > 0:
        start_predictions = []
        
        for _, row in latest_month_for_start.iterrows():
            p = start_predictor.predict(
                row['Next_Kakutei'], row['Next_A'],
                int(row['month']), int(row['day'])
            )
            start_predictions.append({
                'day': row['day'],
                'current': p['current'],
                'forecast': p['forecast'],
                'growth': p['growth'],
                'actual': row.get('Month_End_Next_Start_MP', None)
            })
        
        start_pred_df = pd.DataFrame(start_predictions)
        
        current_ym = latest_month_for_start.iloc[0]['year_month']
        next_month_idx = (int(latest_month_for_start.iloc[0]['month']) % 12) + 1
        mn = ['','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        next_ym_display = mn[next_month_idx]
        
        # 5. 可視化
        plt.figure(figsize=(14, 6))
        plt.plot(start_pred_df['day'], start_pred_df['current']/10000, 
                 marker='o', linewidth=2, color='lightblue', label='Current (Next Kakutei + A)', alpha=0.7)
        plt.plot(start_pred_df['day'], start_pred_df['forecast']/10000, 
                 marker='o', linewidth=2, color='green', label=f'Predicted {current_ym} End Value')
        
        # 実績値があれば表示
        if pd.notna(start_pred_df['actual'].iloc[0]):
            plt.axhline(y=start_pred_df['actual'].iloc[0]/10000, 
                        color='red', linestyle='--', label=f'Actual {current_ym} End Value', linewidth=2)
        
        plt.title(f'Next Month Start MP Prediction: {current_ym} → {next_ym_display} Start')
        plt.xlabel(f'Day of {current_ym}')
        plt.ylabel('Next Month Start MP (10K JPY)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        print(f'\n{next_ym_display} Start MP Prediction:')
        print('='*60)
        print(f'Current ({current_ym} Day {start_pred_df["day"].iloc[-1]}): {start_pred_df["current"].iloc[-1]/10000:>10,.0f} (10K JPY)')
        print(f'Predicted ({current_ym} End):                 {start_pred_df["forecast"].iloc[-1]/10000:>10,.0f} (10K JPY)')
        print(f'Expected Growth:                              {start_pred_df["growth"].iloc[-1]/10000:>10,.0f} (10K JPY)')
        print('='*60)
        print(f'\n💡 Interpretation:')
        print(f'   - By {current_ym} end, {next_ym_display} will start with {start_pred_df["forecast"].iloc[-1]/10000:,.0f} (10K JPY)')
        print(f'   - This is {start_pred_df["growth"].iloc[-1]/10000:,.0f} (10K JPY) more than current value')
    else:
        print('No data available for prediction')
else:
    print('ERROR: No training data for next month start MP prediction!')

In [ ]:
# データ診断：セクション8に必要なデータの確認
print('='*70)
print('DATA DIAGNOSIS FOR SECTION 8')
print('='*70)

# 1. 最新月の確認
print(f'\n1. Latest month in data: {df["year_month"].max()}')
print(f'   Latest date: {df["date"].max().strftime("%Y-%m-%d")}')

# 2. 最新月の日次データ確認
latest_month_data = df[df['year_month'] == df['year_month'].max()]
print(f'\n2. Latest month daily data:')
print(f'   Total days: {len(latest_month_data)}')
print(f'   Non-month-end days: {len(latest_month_data[~latest_month_data["is_month_end"]])}')
print(f'   Date range: {latest_month_data["date"].min().strftime("%Y-%m-%d")} to {latest_month_data["date"].max().strftime("%Y-%m-%d")}')

# 3. 次月データの確認
print(f'\n3. Next month data availability:')
for col in ['Next_Kakutei', 'Next_A', 'Next_B', 'Next_C', 'Next_D']:
    if col in df.columns:
        non_null = latest_month_data[col].notna().sum()
        print(f'   {col}: {non_null}/{len(latest_month_data)} days have data')

# 4. Next_Month_End_MP の確認
print(f'\n4. Next_Month_End_MP availability:')
print(f'   Total records with Next_Month_End_MP: {df["Next_Month_End_MP"].notna().sum()}')
print(f'   Months with Next_Month_End_MP:')
months_with_data = df[df["Next_Month_End_MP"].notna()]["year_month"].unique()
for ym in sorted(months_with_data)[-5:]:  # 直近5ヶ月のみ表示
    print(f'     - {ym}')

# 5. セクション8の条件チェック
print(f'\n5. Section 8 condition check:')
condition1 = df['year_month'] == df['year_month'].max()
condition2 = ~df['is_month_end']
condition3 = df['Next_Month_End_MP'].notna()
combined = condition1 & condition2 & condition3

print(f'   Latest month: {condition1.sum()} records')
print(f'   Non-month-end: {condition2.sum()} records')  
print(f'   Has Next_Month_End_MP: {condition3.sum()} records')
print(f'   ALL conditions met: {combined.sum()} records')

print(f'\n6. Recommendation:')
if combined.sum() > 0:
    print(f'   ✅ Section 8 should work! ({combined.sum()} days available)')
else:
    print(f'   ⚠️  Section 8 will not display (no matching data)')
    print(f'   ')
    print(f'   This is expected for the current month (e.g., Feb 2026)')
    print(f'   because Next_Month_End_MP (Mar 2026 end) doesn\'t exist yet.')
    print(f'   ')
    print(f'   To see Section 8 visualization, use data from a past month')
    print(f'   that has complete next month data.')

print('='*70)